In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch.optim.optimizer import Optimizer
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
from transformers import pipeline
import warnings 
warnings.filterwarnings('ignore')
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader
import os
import gc
gc.collect()

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# set_seed(42)

In [15]:
def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    seed_everything(seed)
    return random_state
random_state = set_seed(42)

Global seed set to 42


# LOAD DATA

In [16]:
df = pd.read_csv('../data/preprocess_for_SQUAD_銀行.csv',index_col=0)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(train_df.shape)
print(val_df.shape)
val_df

(1160, 4)
(291, 4)


,string_X_train,Y_label,string_Y_1,string_Y_2
3174,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND ...,CHINA CITIC BANK,2399,2415
4253,SIGNED COMMERCIAL INVOICE IN 3 COPIESx000DFULL...,KEB HANA BANK,120,133
311,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,492,505
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,2379,2395
870,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,492,505
...,...,...,...,...
7357,nan ALL DOCUMENTS MUST INDICATE THIS LC NOx000...,SAUDI BRITISH BANK,2612,2630
7333,SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL INDICA...,TAISHIN INTERNATIONAL BANK,185,211
4024,1 MANUALLY SIGNED COMMERCIAL INVOICE IN 3 ORIG...,BANK OF CHINA LTD,828,845
1734,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,BANK CENTRAL ASIA,214,231


# Load Model

In [17]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load('../models/Product_Data_SQuAD_model_銀行.pt'))
model.eval()
nlp = pipeline('question-answering', model=model.to('cpu'), tokenizer=tokenizer)
gc.collect()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

94

# rule base

In [18]:
import numpy as np
import re

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 3.去除換行符號
    x = str.strip(x) # 4.移除左右空白
    if 'x000D' in x:
        x = x.replace('x000D','')
    return x

def get_bank(text):
    text = str(text)
    text = preprocess(text)
    keywords = ['TO ORDER OF','TO THEORDER OF','TO THE ORDER OF','TOTHE ORDER OF','TO THE ORDER+OF','TOORDER OF']
    for i in keywords:
        if i in text:
            idx = text.split(i)[1].find('BANK')
            result = preprocess(text.split(i)[1][:idx+len('BANK')])
            if ('BANK' in result) and (len(result) > 4): # 包含BANK且字串超過四個字
                return result
            
            elif str.strip(result) == 'BANK': # 只有BANK當作沒找到
                return 'not find'
            
            else:
                return 'not find'
        else:
            return 'not find'

In [19]:
result = pd.DataFrame()
result['string_X_train'] = val_df['string_X_train']
result['Y_label'] = val_df['Y_label']
result

,string_X_train,Y_label
3174,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND ...,CHINA CITIC BANK
4253,SIGNED COMMERCIAL INVOICE IN 3 COPIESx000DFULL...,KEB HANA BANK
311,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD
870,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD
...,...,...
7357,nan ALL DOCUMENTS MUST INDICATE THIS LC NOx000...,SAUDI BRITISH BANK
7333,SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL INDICA...,TAISHIN INTERNATIONAL BANK
4024,1 MANUALLY SIGNED COMMERCIAL INVOICE IN 3 ORIG...,BANK OF CHINA LTD
1734,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,BANK CENTRAL ASIA


In [20]:
a = val_df['string_X_train'].apply(get_bank)
a = a.dropna(axis=0)
a

3174        not find
4253        not find
311         not find
8266    ISSUING BANK
870         not find
            ...     
7357        not find
7333        not find
4024        not find
1734        not find
7371        not find
Name: string_X_train, Length: 291, dtype: object

In [21]:
result['predict'] = val_df['string_X_train'].apply(get_bank)
result

,string_X_train,Y_label,predict
3174,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND ...,CHINA CITIC BANK,not find
4253,SIGNED COMMERCIAL INVOICE IN 3 COPIESx000DFULL...,KEB HANA BANK,not find
311,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,not find
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,ISSUING BANK
870,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,not find
...,...,...,...
7357,nan ALL DOCUMENTS MUST INDICATE THIS LC NOx000...,SAUDI BRITISH BANK,not find
7333,SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL INDICA...,TAISHIN INTERNATIONAL BANK,not find
4024,1 MANUALLY SIGNED COMMERCIAL INVOICE IN 3 ORIG...,BANK OF CHINA LTD,not find
1734,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,BANK CENTRAL ASIA,not find


In [22]:
result.loc[result['predict'] != 'not find','class'] = 'rule'
result

,string_X_train,Y_label,predict,class
3174,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND ...,CHINA CITIC BANK,not find,NaN
4253,SIGNED COMMERCIAL INVOICE IN 3 COPIESx000DFULL...,KEB HANA BANK,not find,NaN
311,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,not find,NaN
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,ISSUING BANK,rule
870,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,not find,NaN
...,...,...,...,...
7357,nan ALL DOCUMENTS MUST INDICATE THIS LC NOx000...,SAUDI BRITISH BANK,not find,NaN
7333,SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL INDICA...,TAISHIN INTERNATIONAL BANK,not find,NaN
4024,1 MANUALLY SIGNED COMMERCIAL INVOICE IN 3 ORIG...,BANK OF CHINA LTD,not find,NaN
1734,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,BANK CENTRAL ASIA,not find,NaN


# 接bert

In [23]:
not_find = []
for j,i in enumerate(result.iloc[:,2].values):
    if i == 'not find':
        not_find.append(j)
len(not_find)

244

In [24]:
not_find_df = result.iloc[not_find]
not_find_df

,string_X_train,Y_label,predict,class
3174,1SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND ...,CHINA CITIC BANK,not find,NaN
4253,SIGNED COMMERCIAL INVOICE IN 3 COPIESx000DFULL...,KEB HANA BANK,not find,NaN
311,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,not find,NaN
870,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALSx000DC...,MUFG BANK LTD,not find,NaN
3518,1BENEFICIARYS MANUALLY SIGNED COMMERCIAL INVOI...,ASKARI BANK LIMITED,not find,NaN
...,...,...,...,...
7357,nan ALL DOCUMENTS MUST INDICATE THIS LC NOx000...,SAUDI BRITISH BANK,not find,NaN
7333,SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL INDICA...,TAISHIN INTERNATIONAL BANK,not find,NaN
4024,1 MANUALLY SIGNED COMMERCIAL INVOICE IN 3 ORIG...,BANK OF CHINA LTD,not find,NaN
1734,1 SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,BANK CENTRAL ASIA,not find,NaN


In [25]:
def model_predict(nlp,df):
    table = pd.DataFrame()
    for i in tqdm(df.index):
        sample = df.loc[[i]]
        string_X_train = sample['string_X_train'].values[0]
        QA_input = {
            'question': 'What is the bank name?',
            'context': string_X_train
        }
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({'predict:':predict},index=[i])
        table = table.append(row)
    return table

In [26]:
bert_predict = model_predict(nlp,not_find_df)
bert_predict

,predict:
3174,SERVICECHINA CITIC BANKGUANGZHOU
4253,KEB HANA BANKx000DMARKED
311,MUFG BANK LTD
870,MUFG BANK LTD
3518,ASKARI BANK LTD
...,...
7357,SAUDI BRITISH BANK SABBx000DGLOBAL
7333,TAISHIN INTERNATIONAL BANK
4024,BANK OF CHINA LTD
1734,BANK CENTRAL ASIAx000DMARKED


In [27]:
result.loc[bert_predict.index] = bert_predict.values
result

,string_X_train,Y_label,predict,class
3174,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU
4253,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED
311,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,ISSUING BANK,rule
870,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD
...,...,...,...,...
7357,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL
7333,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK
4024,BANK OF CHINA LTD,BANK OF CHINA LTD,BANK OF CHINA LTD,BANK OF CHINA LTD
1734,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED


In [28]:
result.loc[bert_predict.index,'class'] = 'bert'
result

,string_X_train,Y_label,predict,class
3174,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU,bert
4253,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED,bert
311,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD,bert
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,ISSUING BANK,rule
870,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD,bert
...,...,...,...,...
7357,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL,bert
7333,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK,bert
4024,BANK OF CHINA LTD,BANK OF CHINA LTD,BANK OF CHINA LTD,bert
1734,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED,bert


In [29]:
result

,string_X_train,Y_label,predict,class
3174,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU,SERVICECHINA CITIC BANKGUANGZHOU,bert
4253,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED,KEB HANA BANKx000DMARKED,bert
311,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD,bert
8266,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDIC...,CTBC BANK CO LTD,ISSUING BANK,rule
870,MUFG BANK LTD,MUFG BANK LTD,MUFG BANK LTD,bert
...,...,...,...,...
7357,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL,SAUDI BRITISH BANK SABBx000DGLOBAL,bert
7333,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK,TAISHIN INTERNATIONAL BANK,bert
4024,BANK OF CHINA LTD,BANK OF CHINA LTD,BANK OF CHINA LTD,bert
1734,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED,BANK CENTRAL ASIAx000DMARKED,bert


In [30]:
result[result['Y_label']==result['predict']].sample(10).append(result[result['Y_label']!=result['predict']].sample(10))

,string_X_train,Y_label,predict,class
168,CHANG HWA COMMERCIAL BANK LTD,CHANG HWA COMMERCIAL BANK LTD,CHANG HWA COMMERCIAL BANK LTD,bert
5501,1 SIGNED COMMERCIAL INVOICE IN 03 ORIGINALSx00...,VIETINBANK,VIETINBANK,rule
5035,STANDARD CHARTERED BANK,STANDARD CHARTERED BANK,STANDARD CHARTERED BANK,bert
3310,SERVICECHINA CITIC BANKWUHAN,SERVICECHINA CITIC BANKWUHAN,SERVICECHINA CITIC BANKWUHAN,bert
5309,HOURSWETAIPEI FUBON COMMERCIAL BANK,HOURSWETAIPEI FUBON COMMERCIAL BANK,HOURSWETAIPEI FUBON COMMERCIAL BANK,bert
6902,OFx000DFIRST COMMERCIAL BANKx000DNOTIFYx000DCAL,OFx000DFIRST COMMERCIAL BANKx000DNOTIFYx000DCAL,OFx000DFIRST COMMERCIAL BANKx000DNOTIFYx000DCAL,bert
2843,CATHAY UNITED BANK,CATHAY UNITED BANK,CATHAY UNITED BANK,bert
1772,HOURSWETAIPEI FUBON COMMERCIAL BANK,HOURSWETAIPEI FUBON COMMERCIAL BANK,HOURSWETAIPEI FUBON COMMERCIAL BANK,bert
4711,SBI OVERSEAS BRANCHx000DNO65ST,SBI OVERSEAS BRANCHx000DNO65ST,SBI OVERSEAS BRANCHx000DNO65ST,bert
419,VERBUNDVOLKSBANK OWL EGx000DINTERNATIONALES,VERBUNDVOLKSBANK OWL EGx000DINTERNATIONALES,VERBUNDVOLKSBANK OWL EGx000DINTERNATIONALES,bert


In [32]:
def get_acc(df,t=0.75):
    correct = []
    correct_label = []
    for i in df.index:
        jac = get_jaccard_sim(df.loc[i,'Y_label'],df.loc[i,'predict'])
        if jac >= t:
            correct.append('yes')
        else:
            correct.append('no')
    result = pd.Series(correct)
    return result.value_counts()['yes']/len(result)

def get_jac(df):
    all_jacs = []
    for i in df.index:
        all_jacs.append(get_jaccard_sim(str(df.loc[i,'Y_label']),str(df.loc[i,'predict'])))
    return np.sum(all_jacs)/len(all_jacs)

In [33]:
get_acc(result,1),get_acc(result,0.75),get_jac(result)

(0.8694158075601375, 0.872852233676976, 0.9156779902328411)

In [34]:
result.to_csv('../submit/submit_銀行.csv')